In [1]:
from helper import *
from funcs import *

In [2]:
import pandas as pd 

train_data = pd.read_csv('gap-development.tsv', sep='\t')
test_data  = pd.read_csv('gap-test.tsv', sep='\t')

**tf.tsv**

always predict A-coref as True and B-coref as False

In [3]:
f_test = open("gap-test.tsv", "r")
result = open("tf.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    result.write("\t".join([attrs[0]] + ['True', 'False']) + "\n")

f_test.close()
result.close()

In [4]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=tf.tsv

Overall recall: 51.8 precision: 45.9 f1: 48.7
		tp 918	fp 1082
		fn 855	tn 1145
Masculine recall: 51.0 precision: 45.3 f1: 48.0
		tp 453	fp 547
		fn 436	tn 564
Feminine recall: 52.6 precision: 46.5 f1: 49.4
		tp 465	fp 535
		fn 419	tn 581
Bias (F/M): 1.03

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:


**closer.tsv**

Predict closer coreference to pronoun as True

In [5]:
f_test = open("gap-test.tsv", "r")
result = open("closer.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    tf = ['True', 'False']
    if abs(int(attrs[3]) - int(attrs[5])) > abs(int(attrs[3]) - int(attrs[8])):
        tf = ['False', 'True']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [6]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=closer.tsv

Overall recall: 50.1 precision: 44.4 f1: 47.1
		tp 888	fp 1112
		fn 885	tn 1115
Masculine recall: 51.6 precision: 45.9 f1: 48.6
		tp 459	fp 541
		fn 430	tn 570
Feminine recall: 48.5 precision: 42.9 f1: 45.5
		tp 429	fp 571
		fn 455	tn 545
Bias (F/M): 0.94

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:


In [20]:
print_sents(random=True)

development-1590 Hughes captained England for every one of Mercer's seven games in charge, and initially maintained the role when Don Revie was appointed as Ramsey's permanent successor. However, after the first two qualifiers for the 1976 European Championships, Revie decided to drop Hughes from the team. He gave the captaincy to his former Blackpool team-mate Alan Ball, and Hughes appeared only twice for England in 1975.

development-524 The late 1950s to mid-1970s is viewed as the ``golden age'' for New Zealand in the sport and saw Denny Hulme crowned as World Drivers' Championship in 1967. New Zealand drivers have been absent from Formula One events from 1984 with Mike Thackwell to 2017 when Brendon Hartley is expected to make his F1 debut in the 2017 United States Grand Prix.

development-502 He served as flag officer (ashore), Orkney Islands and Shetland Islands, Scotland, and was made Admiral in 1942. He then served as flag officer-in-charge, in Cardiff before retiring from the 

In [8]:
import pandas as pd

In [9]:
train_data = pd.read_csv('gap-development.tsv', sep='\t')
test_data = pd.read_csv('gap-test.tsv', sep='\t')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              2000 non-null   object
 1   Text            2000 non-null   object
 2   Pronoun         2000 non-null   object
 3   Pronoun-offset  2000 non-null   int64 
 4   A               2000 non-null   object
 5   A-offset        2000 non-null   int64 
 6   A-coref         2000 non-null   bool  
 7   B               2000 non-null   object
 8   B-offset        2000 non-null   int64 
 9   B-coref         2000 non-null   bool  
 10  URL             2000 non-null   object
dtypes: bool(2), int64(3), object(6)
memory usage: 144.7+ KB


**subject_pronoun.tsv**

Predict farther coreference pronoun as True when pronoun is subject pronoun (he, she).
Predict closer coreference pronoun as True when pronoun is not subject pronoun, which is object or possessive pronoun(her, his, him).

In [10]:
pronouns = set(train_data['Pronoun'])
pronouns

{'He', 'Her', 'His', 'She', 'he', 'her', 'him', 'his', 'she'}

In [11]:
result = open("subject_pronoun.tsv", 'w')

for attrs in test_data.iloc:
    pro, pro_id = attrs[2:4]
    A, A_id = attrs[4:6]
    B, B_id = attrs[7:9]

    if pro.lower() in ['he', 'she']:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['TRUE', 'FALSE']
        else:
            tf = ['FALSE', 'TRUE']
    else:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['FALSE', 'TRUE']
        else:
            tf = ['TRUE', 'FALSE']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [12]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=subject_pronoun.tsv

Overall recall: 58.3 precision: 51.6 f1: 54.8
		tp 1033	fp 967
		fn 740	tn 1260
Masculine recall: 59.4 precision: 52.8 f1: 55.9
		tp 528	fp 472
		fn 361	tn 639
Feminine recall: 57.1 precision: 50.5 f1: 53.6
		tp 505	fp 495
		fn 379	tn 621
Bias (F/M): 0.96

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:


In [13]:
false_case = 0
for attrs in test_data.iloc:
    if attrs[6] == attrs[9]:
        false_case += 1

false_case/len(test_data)

0.1135

# subject.tsv

draw a relation between two proper nouns with sentence structure

(e.g. Which one is the subject?)

In [14]:
import nltk
sent = "In the next pair, Mikhaylov equalled Grishin's time and they shared Olympic gold."
tagged_sent = nltk.pos_tag(nltk.word_tokenize(sent))

Classify train_data into several categories

1. When words are inside parenthesis
2. When words appear after the pronoun
 * When the pronoun is subject pronoun
 * When the pronoun is object/possessive pronoun
3. When the both coreferences are false
4. When only one word is subject
5. When the both words are subject
6. When none of the words are subject
 * When a word appears multiple times
 * When both word appears same time
    

In [15]:
import pandas as pd 

train_data = pd.read_csv('gap-development.tsv', sep='\t')
test_data  = pd.read_csv('gap-test.tsv', sep='\t')

## 1. When words are inside parenthesis
Assume when word is inside parenthesis, coreference is always false

In [16]:
def isparenthesis(row):
    sent = row['Text']
    A = row['A']
    A_id = row['A-offset']
    B = row['B']
    B_id = row['B-offset']

    return (not (sent[A_id-1] == "(" and sent[A_id+len(A)] == ")"),
            not (sent[B_id-1] == "(" and sent[B_id+len(B)] == ")"))

def only_parenthesis(row):
    return sum(isparenthesis(row)) == 1

In [22]:
parenthesis_data = train_data[train_data.apply(only_parenthesis, axis=1)]

In [23]:
print_sents(datas={'manual':True, 'data':parenthesis_data}, num_lines = 10)

development-946 The Kit*b al-Hayaw*n (**** *******, English: Book of Animals) is a 9th-century Arabic translation of History of Animals: 1--10, On the Parts of Animals: 11--14, and Generation of Animals: 15--19. The book was mentioned by Al-Kind* (d. 850), and commented on by Avicenna (Ibn S*n*) in his The Book of Healing.

development-1765 During the restaurant's 1980s heyday at the top of the Zagat's survey, it was mentioned in Wall Street (1987) by Gordon's call girl when talking to Bud Fox (Charlie Sheen). Mentioned in Crossing Delancey by Isabelle Grossman (Amy Irving) as the place of her intended birthday dinner.

development-1540 Rinkie is about to get married to Vicky (Sharad Kapoor) but she is unaware of his drug dealings. Raju gives Rajesh eight days to break the news to his family and call Raju his son or he'll tell the family himself. After he leaves, Rajesh gets afraid, and hires gangsters to attack him. On his way out, Raju gets beat up by the hired gangsters, though late

result : 22 out of 31 sentences are predicted correct --> accuracy : 71%

## 2. When words appear after the pronoun
Assume coreference of words coming after the pronoun is false

In [19]:
after_pronoun = []

for row in train_data.iloc:
    if row['Pronoun-offset'] < row['A-offset'] or row['Pronoun-offset'] < row['B-offset']:
        after_pronoun.append(row)

after_pronoun = pd.DataFrame(after_pronoun, columns=train_data.columns)

In [20]:
after_subject_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['he', 'she', 'He', 'She'])]
after_other_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['his', 'her', 'him', 'His', 'Her', 'Him'])]

In [21]:
print_sents(datas={'manual':True, 'data':after_subject_pronoun}, num_lines = 10)

development-1603 Amy arrives at the house looking for Esme to find Frank poring over various books and documents, Evelyn sleeping across the room in a wheelchair. Esme arrives not long after Amy, but is surprised to see her. It seems clear that they have not spoken for quite some time, and Esme is so glad to finally see her daughter again that she begins to weep as she hugs Amy. Once she has recovered, we learn that Esme has been acting as a nurse on a television show, and that Frank had been trying to marry Esme practically since the last time Amy had visited.

development-1926 However, Phil rejects Shirley so she apologises to Heather, who is staying with Minty, but she refuses to forgive her friend. Heather is upset when she hears Minty plans to marry Sam Mitchell (Danniella Westbrook), but offers to help buy a ring. However, when Sam reveals she only agreed to marry Minty in order to shut him up, Minty apologises to Heather, and she says when they married it gave her the confidence

In [22]:
def after_pronoun_predict(data):
    A_pred = []
    B_pred = []
    for row in data.iloc:
        A_value = False if row['A-offset'] > row['Pronoun-offset'] else True
        B_value = False if row['B-offset'] > row['Pronoun-offset'] else True

        A_pred.append(A_value)
        B_pred.append(B_value)

    data['A-pred'] = A_pred
    data['B-pred'] = B_pred

In [23]:
after_pronoun_predict(after_subject_pronoun)
after_pronoun_predict(after_other_pronoun)

<ipython-input-22-ceb8dec2fe17>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['A-pred'] = A_pred
<ipython-input-22-ceb8dec2fe17>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['B-pred'] = B_pred


In [24]:
print(accuracy(after_subject_pronoun))
print(accuracy(after_other_pronoun))

0.411214953271028
0.2133676092544987


Result : It doesn't really matter whether a word comes after the pronoun or not

## 3. When the both word coreference is false

Let's find out what they have in common.

In [25]:
false_cases = 0
for row in train_data.iloc:
    if row['A-coref'] == row['B-coref']:
        false_cases += 1

print(false_cases/len(train_data))
print(false_cases)

0.1005
201


There are more than 10 percent of these kind of cases

In [32]:
falses_data = train_data[(train_data['A-coref'] == False) & (train_data['B-coref'] == False)]

In [33]:
print_sents(datas={'manual':True, 'data':falses_data}, num_lines = 10)

development-662 Tom's father, Warwick, turns up and asks his son to lease the family's wine vineyard to a mining company. Maggie tries to stay out of it, but ends up telling Warwick a few home truths. Carlos and Grace each try to help Rob and Colette. Rob meets a surfer on the South Coast and almost has sex with her, before returning home.

development-1642 Elena Blaramberg was born in Orenburg to her Belgian father, a military geodesist Ivan Fyodorovich Blaramberg, who served as a general in the Russian army, and her Greek mother Elena Pavlovna, n*e Mavromikhali. In 1854, her parents moved Elena and her two brothers from Orenburg to Saint Petersburg.

development-245 Before Fanny knew Sasha was back, she hired a professional dancer from San Francisco, CA, ``The Ringer,'' to play ``Clara.'' This creates hostility between the two lead dancers. Michelle helps Ginny, Melanie, and Boo make up from their argument. Boo is still pining for Carl, and Sasha meets a boy from the basketball team 

We can see that there are some patterns in both false sentences

1. There is another pronoun appeared before.
2. when both word is behind the pronoun, and the subject of the sentence is NNP.

1. There is another pronoun appeared before

In [24]:
def prior_pronoun(row):
    token_dict = tokenized_index(row)

    words = token_dict['words']
    word_index = token_dict['word_index']

    male = ['He', 'His', 'Him', 'he', 'his', 'him']
    female = ['She', 'Her', 'she', 'her', 'hers']

    pro_id = word_index[row['Pronoun-offset']]
    A_id = word_index[row['A-offset']]
    B_id = word_index[row['B-offset']]

    if row['Pronoun'] in male:
        for i, word in enumerate(words[:pro_id]):
            if word in male and i < min(A_id, B_id):
                return True
        return False
    elif row['Pronoun'] in female:
        for i, word in enumerate(words[:pro_id]):
            if word in female and i < min(A_id, B_id):
                return True
        return False

    else:
        print("Unexpected pronoun : {}", row['Pronoun'])

In [25]:
prior_pronoun_data = apply_model(train_data, [prior_pronoun], lambda x: (False, False))

1141 0.10429447852760736


In [30]:
def another_subject(row):
    token_dict = tokenized_index(row)

    tagged_sents = token_dict['tagged_sents']
    sents = token_dict['sents']
    word_index = token_dict['word_index']
    sent_index = token_dict['sent_index']

    pro_sent_id = sent_index[word_index[row['Pronoun-offset']]]

    def first_noun(sent):
        for word, tag in sent:
            if tag.startswith('NN') or tag.startswith('PR'):
                return (word, tag)

    word, tag = first_noun(tagged_sents[pro_sent_id])
    if tag.startswith('NNP') and not word in [row['A'].split()[0], row['B'].split()[0]]:
        return True

    return False

In [31]:
def both_latter(row):
    if row['A-offset'] > row['Pronoun-offset'] and row['B-offset'] > row['Pronoun-offset']:
        return True

    return False

mask = train_data.apply(both_latter, axis=1)
both_latter_data = train_data[mask]

In [29]:
another_subject_data = apply_model(train_data, [both_latter, another_subject], lambda x: (False, False))

48 0.5833333333333334


In [33]:
print_sents(datas={'manual':True, 'data':another_subject_data}, num_lines = 5)

development-46 The feature-length documentary traced the journey of the song, Hava Nagila, from the Ukraine to YouTube. Released theatrically and screened at 80 Jewish film festivals, Hava Nagila:The Movie was either opening or closing night at more than half of those festivals. Director Dyanna Taylor's film about her grandmother, the photographer Dorothea Lange, Grab a Hunk of Lightning, aired on PBS's ``American Masters'' in 2014 and was produced by Grossman.

development-1891 We're getting a lot of press for our gay storyline and we like that, but it's nice to think we'll soon see the day when this is no longer a big deal. Meanwhile, on the heels of the announcement of the pending gay storyline in June 2009, Mauceri was replaced as long-running character Carlotta Vega. According to Michael Logan of TV Guide, based on her religious beliefs Mauceri had taken issue with upcoming scenes that would have Carlotta finding a book called How To Tell Your Parents That You're Gay.

development

## 4. When only one word is subject
Assume that subject word is always true, and other word is always false

In [34]:
subject_data = train_data[train_data['Pronoun'].isin(['he', 'she', 'He', 'She'])]
other_data = train_data[train_data['Pronoun'].isin(['his', 'him', 'her', 'His', 'Him', 'Her'])]
print(len(subject_data))
print(len(other_data))

801
1199


In [12]:
row = test_data.iloc[1716]
print(row['Text'], row['A'], row['B'])

The status of the show itself is unknown as it was originally expected to premiere in late 2008, but production has since been pushed back indefinitely. In 2011, Berkley published Ask-Elizabeth, a self-help book for teenage girls which drew from the workshops she conducted for the Ask-Elizabeth program. Berkley Elizabeth


In [35]:
overall_analysis(train_data, [only_subject], [word_subject, latter_word])

100%|██████████| 9/9 [01:31<00:00, 10.17s/it]


,Pronoun,Length,Accuracy0,Accuracy1
0,He,87,0.885057,0.241379
1,She,115,0.921739,0.208696
2,His,18,0.777778,0.388889
3,Her,19,0.736842,0.210526
4,he,139,0.791367,0.374101
5,she,147,0.768707,0.435374
6,his,306,0.767974,0.535948
7,him,58,0.431034,0.534483
8,her,270,0.648148,0.607407
9,total,1159,0.749784,0.458154


## 5. When both words are subject
Assume that latter word is True, while the other is False

In [36]:
both_subject_data = train_data.apply(both_subject, axis=1)
both_subject_data = train_data[both_subject_data]
len(both_subject_data)

202

In [37]:
print_sents(datas={'manual':True, 'data':both_subject_data}, num_lines = 10)

development-191 In order to get hold of Bobby Z after all, Don Huertero takes a colleague of Tad Gruzsa as hostage and proposes an exchange. Bobby Z knows Don Huertero won't rest until he believes him dead. Being worried sick, he bribes Tad Gruzsa. Now Tad Gruzsa conceives a plan to deceive Don Huertero. He wants to make Don Huertero believe Bobby Z was dead without harming the real Bobby Z. When the exchange is supposed to take place, Tad Gruzsa replaces Bobby Z. In his stead the clueless doppelg*nger Tim Kearney (Paul Walker) crosses the border.

development-1076 Fans were appalled by the commercials that ABC put out calling Budig the ``Real'' Greenlee, as well as the clock counting the days until Rebecca's return, which watermarked the screen during Singh's final month of airing. Singh last aired on All My Children on January 15, 2008. Budig returned the next day. After her exit from All My Children, Singh returned to theatre work and appeared in the one woman play The Euthanasiast 

In [38]:
overall_analysis(train_data, [both_subject], [latter_word, former_word, closer_word, farther_word])

100%|██████████| 9/9 [00:35<00:00,  3.94s/it]


,Pronoun,Length,Accuracy0,Accuracy1,Accuracy2,Accuracy3
0,He,10,0.500000,0.400000,0.500000,0.400000
1,She,14,0.571429,0.428571,0.500000,0.500000
2,His,4,0.750000,0.250000,0.750000,0.250000
3,Her,0,0.000000,0.000000,0.000000,0.000000
4,he,18,0.833333,0.055556,0.833333,0.055556
5,she,25,0.640000,0.200000,0.640000,0.200000
6,his,42,0.690476,0.261905,0.690476,0.261905
7,him,22,0.181818,0.681818,0.181818,0.681818
8,her,67,0.597015,0.358209,0.611940,0.343284
9,total,202,0.594059,0.331683,0.594059,0.331683


## 6. When both words are not subject


In [39]:
none_subject_data = train_data.apply(none_subject, axis=1)
none_subject_data = train_data[none_subject_data]
len(none_subject_data)

639

In [40]:
overall_analysis(train_data, [none_subject], [latter_word, former_word, closer_word, farther_word])

100%|██████████| 9/9 [00:34<00:00,  3.86s/it]


,Pronoun,Length,Accuracy0,Accuracy1,Accuracy2,Accuracy3
0,He,41,0.390244,0.463415,0.439024,0.414634
1,She,50,0.280000,0.420000,0.240000,0.460000
2,His,11,0.181818,0.454545,0.181818,0.454545
3,Her,19,0.368421,0.368421,0.368421,0.368421
4,he,78,0.423077,0.371795,0.423077,0.371795
5,she,77,0.337662,0.402597,0.324675,0.415584
6,his,148,0.452703,0.290541,0.506757,0.236486
7,him,18,0.555556,0.222222,0.555556,0.222222
8,her,197,0.502538,0.319797,0.507614,0.314721
9,total,639,0.428795,0.347418,0.441315,0.334898


In [41]:
overall_analysis(train_data, [none_subject, both_latter, another_subject], [lambda x: (False, False)])

100%|██████████| 9/9 [00:41<00:00,  4.60s/it]


,Pronoun,Length,Accuracy0
0,He,0,0.000000
1,She,0,0.000000
2,His,0,0.000000
3,Her,0,0.000000
4,he,2,0.500000
5,she,2,1.000000
6,his,13,0.692308
7,him,0,0.000000
8,her,4,0.750000
9,total,21,0.714286


In [42]:
print_sents(datas={'manual':True, 'data':none_subject_data[none_subject_data.apply(lambda row: row['Pronoun'] in ['He','She','His','Her'], axis=1)]}, num_lines = 10)

development-156 Homer's epics stand at the beginning of the western canon of literature, exerting enormous influence on the history of fiction and literature in general. Alexander the Great (356BC-323BC) was a Greek king of Macedon and the creator of one of the largest empires in ancient history. He was tutored by the philosopher Aristotle and, as ruler, broke the power of Persia, overthrew the Persian king Darius III and conquered the Persian Empire.

development-906 At the Top 24 round as part of Group 1, Jermaine sang ``Steal My Girl'' together with Vanessa Devi, Shanice Hedger and Ruth Matthews as the whole group, and ``Blank Space'' in the solo round, and was sent through to the next round. She sang ``Lay Me Down'' in the Top 16 round and made it through to the finals, singing ``A Sky Full of Stars'' as a group of top 10 finalists and ``Almost Is Never Enough'' in the solo segment of the Identity round.

development-590 Roscoe and his wife had seven sons and three daughters, inclu

1. when two words are close to each other, referred words are more likely to be coreferred word.

 * {True word} (,?) and {Pronoun}


In [43]:
def CC_pronoun(row):
    token_dict = tokenized_index(row)

    tagged_words = token_dict['tagged_words']
    word_index = token_dict['word_index']

    pro_index = word_index[row['Pronoun-offset']]
    A_index = word_index[row['A-offset']]
    B_index = word_index[row['B-offset']]

    def coref(index):
        if index+5 >= len(tagged_words):
            return False
        return (tagged_words[index+1][1] == 'CC' and pro_index == index + 2) or \
               (tagged_words[index+2][1] == 'CC' and pro_index == index + 3)

    return coref(A_index), coref(B_index)

def exist_CC_pronoun(row):
    return sum(CC_pronoun(row)) == 1

In [44]:
apply_model(train_data, [none_subject, exist_CC_pronoun], CC_pronoun)

15 0.9333333333333333


,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL,A-pred,B-pred
478,development-479,Sarah Wadsworth (17 Mar 1649 - ) m. Jonathan A...,his,260,William,222,False,John Wadsworth,242,True,http://en.wikipedia.org/wiki/William_Wadsworth...,False,True
535,development-536,The orchestra moved on to the Mound Club in St...,His,267,George Burns,237,False,Tal Henry,253,True,http://en.wikipedia.org/wiki/Tal_Henry,False,True
607,development-608,Little Fish is a 2005 Australian film directed...,her,276,Liz Watts,222,False,Cate Blanchett,257,True,http://en.wikipedia.org/wiki/Little_Fish_(film),False,True
633,development-634,The royalties from the performances of the wor...,his,310,Frankie Laine,277,False,Paul Weston,296,True,http://en.wikipedia.org/wiki/Jalousie_'Tango_T...,False,True
694,development-695,They moved to Go**bki near Warsaw with their t...,her,289,Krystyna Izbicki,256,False,Anna Zofia,274,True,http://en.wikipedia.org/wiki/Jerzy_and_Irena_K...,False,True
756,development-757,"Immediately after Blackwood's penalty, wide re...",he,332,Theismann,277,False,Garrett,319,True,http://en.wikipedia.org/wiki/Super_Bowl_XVII,False,True
778,development-779,Not only was it the first scholarly edition of...,Her,290,Mary Lascelles,257,True,Jane Austen,274,False,http://en.wikipedia.org/wiki/Jane_Austen,False,True
1212,development-1213,Truth or Dare by Madonna is a lifestyle brand ...,her,321,Material Girl,258,False,Madonna,309,True,http://en.wikipedia.org/wiki/Truth_or_Dare_by_...,False,True
1320,development-1321,"However, Archie Mitchell uses Suzy to plant DN...",she,317,Queen Victoria,278,False,Suzy,308,True,http://en.wikipedia.org/wiki/Suzy_Branning,False,True
1468,development-1469,"He declared that if James were dead, he would ...",his,313,James,232,False,Brokesby,300,True,http://en.wikipedia.org/wiki/Francis_Brokesby,False,True


2. Located in the beggining of a sentence.

In [45]:
def begin_sentence(row):

    token_dict = tokenized_index(row)
    tagged_sents = token_dict['tagged_sents']

    Subjects = []

    def first_NP(sent):

        first_np = ""
        index=0
        while sent[index][1].startswith('NN') or sent[index][1].startswith('W') or sent[index][1] in ["'s", 'DT', 'IN', 'CD']:
            while sent[index][1].startswith('NNP'):
                first_np += sent[index][0] + " "
                if index < len(sent) - 1:
                    index += 1
                else:
                    break

            if len(first_np) > 0:
                Subjects.append(first_np)

            if index < len(sent) - 1:
                index += 1
            else:
                break

    def tagged_split(tagged_sent):
        splitted = []
        index = 0
        
        while(index < len(tagged_sent)):
            try:
                new_index = tagged_sent[index:].index((',',','))
                splitted.append(tagged_sent[index:index+new_index])
                index += new_index+1
            except:
                splitted.append(tagged_sent[index:])
                return splitted

    for sent in tagged_sents:
        for sub_sent in tagged_split(sent):
        
            first_np = first_NP(sub_sent)

    def exist(word):
        for first_word in Subjects:
            if word in first_word:
                return True
            elif first_word in word:
                return True
        return False

    return (exist(row['A']), exist(row['B']))

def begin_sentence_word(row):
    return sum(begin_sentence(row)) == 1

In [64]:
def most_occurance(row):

    token_dict = tokenized_index(row)
    tagged_words = token_dict['tagged_words']

    index=0
    NPs = []
    NP_dict = {}

    while index < len(tagged_words):
        np = ""
        while index < len(tagged_words) and tagged_words[index][1].startswith('NNP'):
            np += tagged_words[index][0] + " "
            index += 1
        
        if len(np) > 0:
            NPs.append(np[:-1])

        index += 1

    def occurance(word):
        occ = 0
        for w in NPs:
            if w in word:
                occ += 1

        return occ

    for word in set(NPs):
        NP_dict[word] = occurance(word)

    max_occ = sorted(NP_dict.items(), key = (lambda x: x[1]), reverse=True)[0]

    if max_occ[1] > 1:
        return [NP for NP in NP_dict if NP_dict[NP] == max_occ[1]]
    else:
        return None

def more_occurance(row):

    most_occs = most_occurance(row)

    if most_occs == None:
        return (True, True)

    def exist(word, words):
        for w in words:
            if word in w or w in word:
                return True
        return False

    return (exist(row['A'], most_occs), exist(row['B'], most_occs))

def only_most_occurance(row):
    return sum(more_occurance(row)) == 1

def none_most_occurance(row):
    return sum(more_occurance(row)) == 0

def both_most_occurance(row):
    return more_occurance(row) == (True, True)

In [46]:
overall_analysis(train_data, [none_subject, only_most_occurance], [more_occurance])

100%|██████████| 9/9 [00:22<00:00,  2.51s/it]


,Pronoun,Length,Accuracy0
0,He,20,0.700000
1,She,16,0.562500
2,His,4,0.750000
3,Her,9,0.555556
4,he,26,0.653846
5,she,34,0.441176
6,his,59,0.576271
7,him,9,0.555556
8,her,92,0.597826
9,total,269,0.583643


In [47]:
none_occurance_data = none_subject_data[none_subject_data.apply(none_most_occurance, axis=1)]

In [48]:
both_occurance_data = none_subject_data[none_subject_data.apply(both_most_occurance, axis=1)]

In [49]:
print(len(none_occurance_data))
print(len(both_occurance_data))

200
170


In [103]:
print_sents(datas={'manual':True, 'data':both_occurance_data[both_occurance_data.apply(lambda row: row['Pronoun'] in ['He','She','His','Her'], axis=1)]}, num_lines = 10)

development-1487 New York Times. May 7, 1947. Retrieved 2014-12-02. Mme. Louise Homer, noted contralto, who thrilled Metropolitan Opera audiences as Amneris in 'Aida' and in other famous roles, died here this afternoon of a heart ailment at the age of 76. She had been in ill health since the death of her youngest daughter, Joy, on Oct.

development-962 After Dark Shadows ended in 1971, she had a brief stint as reporter Marge Grey on All My Children (1973). She continued acting on stage as Warda in Jean Genet's The Screens (1971--72) and The Lady in Gray/The Fly in Happy End (1977) which co-starred Meryl Streep and Christopher Lloyd. She appeared in the Broadway premiere of The Suicide (1980) with Derek Jacobi, and appeared opposite Geraldine Page, Carrie Nye and Madeline Sherwood in an off-Broadway revival of The Madwoman of Chaillot.

development-1230 Ang Lalaking Nagmahal Sa Akin by Vanessa, a romance writer from PHR, is the story of Flor who moves from the province to the city to in

In [106]:
overall_analysis(train_data, [none_subject, both_most_occurance], [former_word, latter_word, closer_word, farther_word, lambda row: (False, False)])

100%|██████████| 9/9 [00:18<00:00,  2.10s/it]


,Pronoun,Length,Accuracy0,Accuracy1,Accuracy2,Accuracy3,Accuracy4
0,He,9,0.555556,0.444444,0.444444,0.555556,0.000000
1,She,21,0.428571,0.142857,0.142857,0.428571,0.428571
2,His,2,0.500000,0.000000,0.000000,0.500000,0.500000
3,Her,3,0.000000,1.000000,1.000000,0.000000,0.000000
4,he,23,0.478261,0.217391,0.260870,0.434783,0.304348
5,she,20,0.400000,0.400000,0.350000,0.450000,0.200000
6,his,39,0.128205,0.615385,0.564103,0.179487,0.256410
7,him,3,0.000000,0.666667,0.666667,0.000000,0.333333
8,her,50,0.280000,0.560000,0.560000,0.280000,0.160000
9,total,170,0.311765,0.452941,0.441176,0.323529,0.235294


In [56]:
apply_model(train_data, [none_subject, both_most_occurance, lambda row: row['Pronoun'] in ['Her','his','him','her']], latter_word)

95 0.6


,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL,A-pred,B-pred
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,http://en.wikipedia.org/wiki/Crime_(band),False,True
6,development-7,Reb Chaim Yaakov's wife is the sister of Rabbi...,his,273,Reb Asher,152,False,Akiva Eiger,253,False,http://en.wikipedia.org/wiki/Asher_Arieli,False,True
10,development-11,William Shatner portraying writer Mark Twain; ...,his,300,Peter Mansbridge,168,False,David Onley,212,True,http://en.wikipedia.org/wiki/Murdoch_Mysteries,False,True
13,development-14,ARTA driver Vitantonio Liuzzi will be replaced...,his,340,Kazuki Nakajima,275,True,Oliver Jarvis,297,False,http://en.wikipedia.org/wiki/2015_Super_GT_Series,False,True
25,development-26,"They were broadly positive, though they stoppe...",his,299,Michael Billington,92,False,Martin,266,True,http://en.wikipedia.org/wiki/Life_and_Beth,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1923,development-1924,The concept of WAS is an adult parallel to the...,her,267,Dorothy Gael,169,True,Aunt Emily,234,False,http://en.wikipedia.org/wiki/Was_(novel),False,True
1935,development-1936,He also wrote and produced several songs for i...,her,266,Estelle,162,False,Nelly Furtado,190,True,http://en.wikipedia.org/wiki/Di_Genius,False,True
1976,development-1977,``Don't Cry Your Heart Out'' was radio-friendl...,her,320,Deasismont,206,False,Amy,312,True,http://en.wikipedia.org/wiki/Amy_Deasismont,False,True
1988,development-1989,Burnett Stone (Peter Fonda) is Lily's grandfat...,her,81,Lily,31,True,Lady,54,False,http://en.wikipedia.org/wiki/Thomas_the_Tank_E...,False,True


In [113]:
print_sents(datas={'manual':True, 'data':none_occurance_data}, num_lines = 10)

development-1474 Later Glenn helps Michonne cut off Tyreese's arm when he is bitten and tries to haul him to safety, briefly being delayed by a truck opening and letting out numerous walker bodies with no arms and legs with ``w''s cut into their foreheads, but Tyreese dies from blood loss and Glenn stands over his service. In the episode ``Them,'' Glenn tries to comfort a grieving Maggie, still distraught over Beth's death, after she begins to doubt if she wants to live anymore.

development-1299 In 1920, Jakac returned to Novo Mesto and became the bearer of The Spring of Novo Mesto, an avant-garde movement in literature and fine arts, which included also the poets Miran Jarc and Anton Podbev*ek, the painter Ivan *argo and the composer Marij Kogoj. In 1924, he settled in Ljubljana.

development-1619 St John's Church has special association with the activity of individuals within the South Sea Islander community and Anglican Church whose activities have been significant within the histo

In [105]:
overall_analysis(train_data, [none_subject, none_most_occurance], [former_word, latter_word, closer_word, farther_word, lambda row: (False, False)])

100%|██████████| 9/9 [00:19<00:00,  2.12s/it]


,Pronoun,Length,Accuracy0,Accuracy1,Accuracy2,Accuracy3,Accuracy4
0,He,12,0.333333,0.250000,0.250000,0.333333,0.416667
1,She,13,0.461538,0.230769,0.153846,0.538462,0.307692
2,His,5,0.600000,0.000000,0.000000,0.600000,0.400000
3,Her,7,0.285714,0.142857,0.142857,0.285714,0.571429
4,he,29,0.379310,0.379310,0.379310,0.379310,0.241379
5,she,23,0.391304,0.304348,0.260870,0.434783,0.304348
6,his,50,0.260000,0.340000,0.420000,0.180000,0.400000
7,him,6,0.166667,0.500000,0.500000,0.166667,0.333333
8,her,55,0.418182,0.345455,0.345455,0.418182,0.236364
9,total,200,0.360000,0.320000,0.330000,0.350000,0.320000


In [112]:
rest = apply_model(train_data, [none_subject, none_most_occurance], begin_sentence)

200 0.35


In [5]:
overall_analysis(train_data, [], [subject])

100%|██████████| 9/9 [02:59<00:00, 19.91s/it]


,Pronoun,Length,Accuracy0
0,He,138,0.775362
1,She,179,0.754190
2,His,33,0.696970
3,Her,38,0.578947
4,he,235,0.629787
5,she,249,0.610442
6,his,496,0.663306
7,him,98,0.459184
8,her,534,0.573034
9,total,2000,0.633500


In [4]:
overall_analysis(test_data, [], [subject])

100%|██████████| 10/10 [03:22<00:00, 20.24s/it]


,Pronoun,Length,Accuracy0
0,He,127,0.811024
1,She,159,0.716981
2,His,28,0.535714
3,Her,37,0.648649
4,he,221,0.633484
5,she,237,0.649789
6,his,528,0.619318
7,him,96,0.447917
8,her,566,0.570671
9,hers,1,0.000000


In [5]:
subject_data = apply_model(test_data, [], subject)

2000 0.6215


In [ ]:
result = open("subject_word.tsv", 'w')

for row in subject_data.iloc:

    result.write("\t".join([attrs[0]] + row['A-pred'] + row['B-pred']) + "\n")

f_test.close()
result.close()

In [21]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=subject_word.tsv

Overall recall: 68.5 precision: 63.3 f1: 65.8
		tp 1215	fp 704
		fn 558	tn 1523
Masculine recall: 68.7 precision: 64.2 f1: 66.4
		tp 611	fp 340
		fn 278	tn 771
Feminine recall: 68.3 precision: 62.4 f1: 65.2
		tp 604	fp 364
		fn 280	tn 752
Bias (F/M): 0.98

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
